<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/BERT_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT Based Sentiment Analysis <br>
*original is at: https://www.kaggle.com/pastrop/toxic-data-comp-data*

In [ ]:
!pip install bert-for-tf2

In [ ]:
#from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import tensorflow_hub as hub
#import bert

In [ ]:
import pandas as pd
import os, time

In [ ]:
print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)

TensorFlow Version: 2.2.0
Hub version:  0.8.0


In [ ]:
#TPU hook-up
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Running on TPU  ['10.125.117.122:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.125.117.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.125.117.122:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
df = pd.read_csv('path to file')
df.head(5)

In [ ]:
# file upload while using Google Colab
from google.colab import files
uploaded = files.upload()

Saving jigsaw-unintended-bias-train-filtered-seqlen256.csv.zip to jigsaw-unintended-bias-train-filtered-seqlen256.csv.zip


In [ ]:
!unzip jigsaw-unintended-bias-train-filtered-seqlen256.csv.zip

Archive:  jigsaw-unintended-bias-train-filtered-seqlen256.csv.zip
  inflating: jigsaw-unintended-bias-train-filtered-seqlen256.csv  
  inflating: __MACOSX/._jigsaw-unintended-bias-train-filtered-seqlen256.csv  


In [ ]:
wiki_toxic_comment_data = "jigsaw-toxic-comment-train.csv" #training file to be processed
wiki_toxic_comment_train = pd.read_csv(os.path.join(DATA_PATH, wiki_toxic_comment_data))
wiki_toxic_comment_train.head()

# Initial Dataset Processing - Adding word_ids, mask_ids, segment_ids

In [ ]:
#building the tokenizer
#bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_cased_L-24_H-1024_A-16/2",trainable=True) - English only
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/2",trainable=True) # Multi-language
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

In [ ]:
#examples of using the tokenizer (not needed for production code, useful to checking that tokenizer is working correctly)
example_sentence = wiki_toxic_comment_train.iloc[37].comment_text[:150]
print(example_sentence)

example_tokens = tokenizer.tokenize(example_sentence)
print(example_tokens[:17])

example_input_ids = tokenizer.convert_tokens_to_ids(example_tokens)
print(example_input_ids[:17])

This is a where the majority of the data prep time is spent. Every record in the input has to be processed. I believe that the process is highly parallezible. The steps are: (1) read file into memory (2) process every record (3) write back to storage. As an example, processing 230,000 records file takes ~600 seconds (Standard Kaggle compute resource). In theory step (3) may be avoided if you have enough memory to hold results in memory for future use yet input files my be arbitrarily large and current impementation of the Tensor Flow dataset class mandates input from the CSV file (not sure why...). This step is required for pretty much all modern NLP models and needs to be performed both in training and production

In [ ]:
def process_sentence(sentence, max_seq_length=SEQUENCE_LENGTH, tokenizer=tokenizer):
    """Helper function to prepare data for BERT. Converts sentence input examples
    into the form ['input_word_ids', 'input_mask', 'segment_ids']."""
    # Tokenize, and truncate to max_seq_length if necessary.
    tokens = tokenizer.tokenize(sentence)
    if len(tokens) > max_seq_length - 2:
        tokens = tokens[:(max_seq_length - 2)]

    # Convert the tokens in the sentence to word IDs.
    input_ids = tokenizer.convert_tokens_to_ids(["[CLS]"] + tokens + ["[SEP]"])

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    pad_length = max_seq_length - len(input_ids)
    input_ids.extend([0] * pad_length)
    input_mask.extend([0] * pad_length)

    # We only have one input segment.
    segment_ids = [0] * max_seq_length

    return (input_ids, input_mask, segment_ids)

def preprocess_and_save_dataset(unprocessed_filename, text_label='comment_text',
                                seq_length=SEQUENCE_LENGTH, verbose=True):
    """Preprocess a CSV to the expected TF Dataset form for multilingual BERT,
    and save the result."""
    dataframe = pandas.read_csv(os.path.join(DATA_PATH, unprocessed_filename),
                                index_col='id')
    processed_filename = (unprocessed_filename.rstrip('.csv') +
                          "-processed-seqlen{}.csv".format(SEQUENCE_LENGTH))

    pos = 0
    start = time.time()

    while pos < len(dataframe):
        processed_df = dataframe[pos:pos + 10000].copy()

        processed_df['input_word_ids'], processed_df['input_mask'], processed_df['segment_ids'] = (
            zip(*processed_df[text_label].apply(process_sentence)))

        if pos == 0:
            processed_df.to_csv(processed_filename, index_label='id', mode='w')
        else:
            processed_df.to_csv(processed_filename, index_label='id', mode='a',
                                header=False)

        if verbose:
            print('Processed {} examples in {}'.format(
                pos + 10000, time.time() - start))
        pos += 10000
    return
  
# Process the training dataset.
preprocess_and_save_dataset(wiki_toxic_comment_data)

# Dataset transformation into tf.dataset

In [ ]:
#CONSTANTS
SEQUENCE_LENGTH = 256

The below data transformations run fast

In [ ]:
df = pd.read_csv('/content/jigsaw-unintended-bias-train-filtered-seqlen256.csv', nrows = 10) # Limit nrows for the test run, read the entire  file otherwise
df.head()
#df_filter = df.filter(['toxic','input_word_ids','input_mask','segment_ids'])
#df_filter.head(3)

,toxic,input_word_ids,input_mask,segment_ids
0,0.000000,"(101, 10747, 10124, 10380, 67420, 119, 10377, ...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,0.000000,"(101, 91327, 13028, 106, 106, 10747, 10894, 13...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,0.000000,"(101, 10747, 10124, 11049, 10151, 10399, 22500...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,0.000000,"(101, 12034, 10531, 26133, 146, 112, 22469, 10...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,0.893617,"(101, 10228, 10921, 13028, 75980, 12682, 10301...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
df_filter.to_csv('train_toxic_256.csv', index = False, mode='w')

*tf.dataset creation (depends on how the file was pre-processed)*

In [ ]:
#test = df.filter(['toxic','input_word_ids','input_mask','all_segment_id'])
#test = test.rename(columns={"all_segment_id": "segment_ids"})
#test.head(3)

In [ ]:
# writing dataframe to CSV
#test.to_csv('test_processed_256.csv', index = False, mode='w')

In [ ]:
#building TF dataset:
def get_dataset(file_path = 'path to file'):
    dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=12, # Artificially small, the dataset is batched up later.
      label_name='toxic', #label for the class column
      na_value="?",
      num_epochs=1,
      shuffle=False)
    return dataset

In [ ]:
file_path = '/content/jigsaw-unintended-bias-train-filtered-seqlen256.csv'
train_data = get_dataset(file_path)
train_data = train_data.unbatch()

In [ ]:
def parse_string_list_into_ints(strlist):
    s = tf.strings.strip(strlist)
    s = tf.strings.substr(s, 1, tf.strings.length(s) - 2)  # Remove parentheses around list
    #s = tf.strings.split(s, ',', maxsplit=128)
    s = tf.strings.split(s, ',', maxsplit=256)
    s = tf.strings.to_number(s, tf.int32)
    #s = tf.reshape(s, [128])  # Force shape here needed for XLA compilation (TPU)
    s = tf.reshape(s, [256])  # Force shape here needed for XLA compilation (TPU)
    return s

In [ ]:
# prototype function to process the dataset for the Bert layer
def elem_mod(data,label):
    for k,v in data.items():
        data[k] = parse_string_list_into_ints(v)
    return data,label    
    
result = train_data.map(lambda x,y:elem_mod(x,y))

In [ ]:
#Final step in getting the dataset ready:
def make_dataset_pipeline(dataset, repeat_and_shuffle=True):
    """Set up the pipeline for the given dataset.   
    Caches, repeats, shuffles, and sets the pipeline up to prefetch batches."""
    cached_dataset = dataset.cache()
    if repeat_and_shuffle:
        cached_dataset = cached_dataset.shuffle(2048)
    #cached_dataset = cached_dataset.batch(32 * strategy.num_replicas_in_sync)
    cached_dataset = cached_dataset.batch(32)
    cached_dataset = cached_dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return cached_dataset

In [ ]:
cached_train_data = make_dataset_pipeline(result)

# Keras Model with BERT Layer

In [ ]:
#Building the model (reformat as a function...)
max_seq_length = 256  # Your choice here.
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")
# BERT layer from pretrained model
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/2",trainable=True)
# Dense Layers
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
output = tf.keras.layers.Dense(32, activation='relu')(pooled_output)
output = tf.keras.layers.Dense(1, activation='sigmoid', name='labels')(output)

In [ ]:
# Model
model = tf.keras.Model(inputs={'input_word_ids': input_word_ids,
                                  'input_mask': input_mask,
                                  'all_segment_id': segment_ids},
                          outputs=output)

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
    metrics=[tf.keras.metrics.AUC()])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 256)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 256)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 177853441   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

*Model Build&Compile if TPU is used*

In [ ]:
#TPU based model DON'T RUN WITHOUT TPU
# Model for TPU distribution:
with tpu_strategy.scope():
    max_seq_length = 256  # Your choice here.
    input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                           name="input_word_ids")
    input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_mask")
    segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                        name="segment_ids")
    # BERT layer from pretrained model
    bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/2",trainable=True)
    # Dense Layers
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    output = tf.keras.layers.Dense(32, activation='relu')(pooled_output)
    output = tf.keras.layers.Dense(1, activation='sigmoid', name='labels')(output)
    # Model
    model = tf.keras.Model(inputs={'input_word_ids': input_word_ids,
                                      'input_mask': input_mask,
                                      'all_segment_id': segment_ids},
                              outputs=output)

    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
        metrics=[tf.keras.metrics.AUC()])

    model.summary()


In [ ]:
# Train on English Wikipedia comment data.
history = model.fit(
    # Set steps such that the number of examples per epoch is fixed.
    # This makes training on different accelerators more comparable.
    cached_train_data,steps_per_epoch=4000//256,
    epochs=7, verbose=1)
#print()
#steps_per_epoch=4000//256
#validation_data=nonenglish_val_datasets['Combined'],
# validation_steps=100

In [ ]:
results = model.evaluate(cached_vaidate_data,
                                     steps=100, verbose=0)
print('\nLoss, AUC before training:', results) 

# SANDBOX

In [ ]:
df = pd.read_csv('jigsaw-unintended-bias-train-filtered-seqlen256.csv', nrows=100)
df.head()

,toxic,input_word_ids,input_mask,segment_ids
0,0.000000,"(101, 10747, 10124, 10380, 67420, 119, 10377, ...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,0.000000,"(101, 91327, 13028, 106, 106, 10747, 10894, 13...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,0.000000,"(101, 10747, 10124, 11049, 10151, 10399, 22500...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,0.000000,"(101, 12034, 10531, 26133, 146, 112, 22469, 10...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,0.893617,"(101, 10228, 10921, 13028, 75980, 12682, 10301...","(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
target = df.pop('toxic')

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), target.values))# it may work to get the same dataset structure as tf.expiremental!

In [ ]:
next(iter(dataset))

({'input_mask': <tf.Tensor: shape=(), dtype=string, numpy=b'(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)'>,
  'input_word_ids': <tf.Tensor: shape=(), dtype=string, numpy=b'(101, 10747, 10124, 10380, 67420, 119, 10377, 112, 187, 11850, 117, 112, 10894, 13028, 21528, 20442, 152

In [ ]:
df.to_csv('test.csv', index = False, mode='w') #test writing into local storage